# Decorators

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


We've seen Python functions and objects.  Today, we'll breifly cover [decorators](https://www.geeksforgeeks.org/decorators-in-python/), which are functions that wrap functions, and come with [sytactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar) which lets you write things like

```python
@time_wrapper
def my_function(x):
    ...
    
y = my_function(x)
```
```
> 1.1 seconds elapsed.
```

Again, we want to write a function that wraps another function.  We can define this as

In [4]:
from time import monotonic

def time_wrapper(f):
    
    def inner_wrapper(*args, **kwargs):
        t0 = monotonic()
        ret = f(*args, **kwargs)
        t1 = monotonic()
        print("{} sec. elapsed".format(t1 - t0))
        return ret
    
    return inner_wrapper

Here's how we might use our wrapper:

In [6]:
def generate_random(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random = time_wrapper(generate_random)

x = generate_random(100000)
len(x)

0.001988777003134601 sec. elapsed


100000

one problem with this is that we had to go through the trouble of wrapping the function. Instead, we can just write

In [7]:
@time_wrapper
def generate_random2(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random2(100000)

0.001884669007267803 sec. elapsed


array([0.98937031, 0.08336035, 0.45845972, ..., 0.53122849, 0.06449548,
       0.91733102])

This is interpreted like what we did above, but is a bit easier to read.

Another problem we may encounter is that we now can't access the docstring we wrote

In [8]:
help(generate_random2)

Help on function inner_wrapper in module __main__:

inner_wrapper(*args, **kwargs)



One way to solve this is to use the `wraps` decorator from the [functools](https://docs.python.org/3/library/functools.html) package.

In [10]:
from functools import wraps

def time_wrapper2(f):
    
    @wraps(f)
    def inner_wrapper(*args, **kwargs):
        t0 = monotonic()
        ret = f(*args, **kwargs)
        t1 = monotonic()
        print("{} sec. elapsed".format(t1 - t0))
        return ret
    
    return inner_wrapper

@time_wrapper2
def generate_random3(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random3(100000)

0.001709334013867192 sec. elapsed


array([0.72765311, 0.62323703, 0.53931556, ..., 0.41920835, 0.66975116,
       0.30342516])

now, we can access the metadata from `generate_random3`, such as the docstring

In [11]:
help(generate_random3)

Help on function generate_random3 in module __main__:

generate_random3(n)
    return a random numpy vector



As you might guess, the `@wraps` decorator copies the class metadata from `f` to the `inner_wrapper` function.